In [1]:
!pip install -q llama-index langchain
!pip install -q llama-index-embeddings-huggingface

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf

In [2]:
from llama_index.core import Document, Settings, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index.core.node_parser import LangchainNodeParser

In [3]:
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

Settings.embed_model = embed_model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import json
narrations = r'/kaggle/input/fyp-dataset-ego4d/narrations.json'

with open(narrations, 'r') as file:
    data = json.load(file)
    
videos = list(data.keys())

In [37]:
ground_truths = {'8d900b59-e95d-4c9b-9432-8f5096aac2db': [[6000, 8700], [9810, 10710]],
                 'e2469ed9-64a2-42b4-8cf7-089b1c7e7ba5': [[2760, 7000]],
                 '61767972-9303-4f55-980c-832d7a454d6e': [[8310, 11000]],
                 'fd878068-4a9a-458d-9225-fe4865907f0a': [[60, 2760]],
                 '3c6ac251-4298-4421-be93-9262bcdbc320': [[3600, 4950], [8550, 11610]]}

In [38]:
def read_dictionary_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            # Read the contents of the file
            data = file.read()
            # Convert the string representation of dictionary to a dictionary object
            dictionary = eval(data)
            return dictionary
    except FileNotFoundError:
        print("File not found.")
    except Exception as e:
        print("An error occurred:", e)

In [39]:
file_paths = ['/kaggle/input/fyp-dataset-ego4d/8d900b59-e95d-4c9b-9432-8f5096aac2db.txt',
              '/kaggle/input/fyp-dataset-ego4d/e2469ed9-64a2-42b4-8cf7-089b1c7e7ba5.txt',
              '/kaggle/input/fyp-dataset-ego4d/61767972-9303-4f55-980c-832d7a454d6e.txt',
              '/kaggle/input/fyp-dataset-ego4d/fd878068-4a9a-458d-9225-fe4865907f0a.txt',
              '/kaggle/input/fyp-dataset-ego4d/3c6ac251-4298-4421-be93-9262bcdbc320.txt']

In [40]:
preds = []
for file in file_paths:
    preds.append(read_dictionary_from_file(file))

In [41]:
def get_Documents(pred):
    documents = []
    for frame, caption in pred.items():
        documents.append(Document(
            text=caption[0]['generated_text'],
            metadata={
                'frame': int(frame)
            },
            metadata_template="{key}=>{value}",
        ))
    return documents

In [42]:
documents = []
for pred in preds:
    documents.append(get_Documents(pred))
len(documents)

5

In [43]:
parser = LangchainNodeParser(RecursiveCharacterTextSplitter())

In [44]:
nodes = []
for doc in documents:
    nodes.append(parser.get_nodes_from_documents(doc))

In [45]:
retriever = []
for node in nodes:
    retriever.append(VectorStoreIndex(node).as_retriever(similarity_top_k=60))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [46]:
responses = []
responses.append(retriever[0].retrieve('dishes'))
responses.append(retriever[1].retrieve('laundry'))
responses.append(retriever[2].retrieve('kitchen'))
responses.append(retriever[3].retrieve('vaccuming stairs'))
responses.append(retriever[4].retrieve('hanging'))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
from tqdm import tqdm
#response = retriever.retrieve('laundry')
all_similar_frames = []
for response in responses:    
    similar_frames = []
    for r in response:
        #print(r.text, r.metadata)
        similar_frames.append(r.metadata['frame'])
    similar_frames = sorted(similar_frames)
    
    i = 1
    while i < len(similar_frames):
        if similar_frames[i] <= similar_frames[i - 1] + 120:
            del similar_frames[i]
        else:
            i += 1
    i = 0
    for i in tqdm(range(0,len(similar_frames)-1)):
        forward_diff = similar_frames[i] + 120
        backward_diff = similar_frames[i + 1] - 120
        if forward_diff > backward_diff:
            difference = abs((similar_frames[i+1]) - similar_frames[i] + 120)
            similar_frames[i+1] += difference + 120
        i += 1
        
    similar_frames = list(dict.fromkeys(similar_frames))
    all_similar_frames.append(similar_frames)

100%|██████████| 25/25 [00:00<00:00, 181414.53it/s]


In [48]:
def makeFrames(framesList, typee = 's'):
    frames = []
    if typee == 'gt':
        for i in framesList:
            start, end = i[0], i[1]
            frames += list(range(start,  end  + 1))
    else:
        for i in framesList:
            frames += list(range(i - 120,  i + 120  + 1))
    return frames

In [49]:
def Evaluate(ground_truth, machine_summary):
    '''
    ground_truth = ground truth summary frames
    machine_summary = machine generated summary frames
    '''
    true_positives = [frame for frame in machine_summary if frame in ground_truth]
    false_positives = [frame for frame in machine_summary if frame not in ground_truth]
    false_negatives = [frame for frame in ground_truth if frame not in machine_summary]
    
    precision = len(true_positives) / (len(true_positives) + len(false_positives))
    recall = len(true_positives) / (len(true_positives) + len(false_negatives))
    
    f1 = (2 * precision * recall) / (precision + recall)
    return f1, precision, recall

In [50]:
all_ground_truths = []
for video, gt in ground_truths.items():
    all_ground_truths.append(makeFrames(gt, 'gt'))

In [51]:
all_summaries = []
for summary in all_similar_frames:
    all_summaries.append(makeFrames(summary))

In [52]:
f1_scores, precision, recall = [], [], []
for i in range(0, 5):
    f1, prec, rec = Evaluate(all_ground_truths[i], all_summaries[i])
    f1_scores.append(f1)
    precision.append(prec)
    recall.append(rec)

In [53]:
print("F1-score: ", sum(f1_scores)/5)

F1-score:  0.4773835686893054


In [54]:
print("Recall: ", sum(recall)/5)

Recall:  0.5736577991302197


In [55]:
f1_scores

[0.41747871576571954,
 0.7134071340713407,
 0.19792629093522227,
 0.6006169909824395,
 0.45748871169180516]

In [56]:
recall

[0.5464554242749732,
 0.7050243111831442,
 0.3502906976744186,
 0.7517077517077517,
 0.5148108108108108]

In [57]:
precision

[0.3377593360995851,
 0.7219917012448133,
 0.13793103448275862,
 0.5000987947046038,
 0.4116528354080221]